## Problem: Stock Trading Bot - Maximize Revenue

Date: 9/17/2025

**Setup:**
You're creating a bot to handle stock trades for ONE stock. Each day, the bot either buys or sells one share.

**Given:**
- `prices`: array of positive integers where `prices[i]` = stock price on day i
- `algo`: array of 0s and 1s where:
  - `algo[i] = 0` means **buy** on day i
  - `algo[i] = 1` means **sell** on day i

**Goal:**
Choose a range of **k consecutive days** from `algo` to modify/override, such that it **maximizes total revenue**.

**Revenue calculation:**
- Revenue = sum of all selling prices - sum of all buying prices
- Start with 0 revenue and enough shares to always sell when needed

**Task:**
Return the maximum possible revenue after optimally choosing which k consecutive days to override.

---

**Example:**
```
prices = [2, 4, 1, 5, 2, 6, 7]
algo = [0, 1, 0, 0, 1, 0, 0]
k = 4
```

**Output:** `21`

**Walkthrough of default algo (no override):**
```
Day 0: algo[0]=0 → BUY at price 2  → revenue = 0 - 2 = -2
Day 1: algo[1]=1 → SELL at price 4 → revenue = -2 + 4 = 2
Day 2: algo[2]=0 → BUY at price 1  → revenue = 2 - 1 = 1
Day 3: algo[3]=0 → BUY at price 5  → revenue = 1 - 5 = -4
Day 4: algo[4]=1 → SELL at price 2 → revenue = -4 + 2 = -2
Day 5: algo[5]=0 → BUY at price 6  → revenue = -2 - 6 = -8
Day 6: algo[6]=0 → BUY at price 7  → revenue = -8 - 7 = -15

Total revenue = -15
```

With optimal k=4 window override, we can get revenue = 21.

---

## Initial Analysis

This is a **sliding window + greedy optimization** problem. Key insights:

1. **We can override ANY k consecutive days** - so there are `n - k + 1` possible windows to try
2. **Within the chosen window, we want the optimal strategy** - likely all sells at high prices, all buys at low prices
3. **Outside the window, we follow the original algo**

The tricky part: What's the "optimal" strategy within the k-day window?

**Intuition:** 
- Within our k-day window, we want to **sell on high-price days** and **buy on low-price days**
- But we're constrained by having to do one action per day

Let me think about this more carefully...

Actually, I think the problem is asking: **choose k consecutive days where you can REWRITE the algo array optimally**.

So the approach:
1. Try all possible k-length windows `[i, i+k)`
2. For each window, figure out the optimal buy/sell strategy for those k days
3. Calculate total revenue: (before window) + (optimal window) + (after window)
4. Return maximum

Want to think through what "optimal strategy for k days" means? Or should I walk you through it?

In [ ]:
def solution(prices, algo, k):
    n = len(prices)
    base = 0
    for i in range(n):
        base += prices[i] if algo[i] == 1 else -prices[i]
    
    # Gain from flipping buy day to sell day = 2*price
    gain = 0
    for i in range(min(k, n)):
        if algo[i] == 0:
            gain += 2 * prices[i]
    
    best = gain
    for i in range(k, n):
        if algo[i] == 0:
            gain += 2 * prices[i]
        if algo[i - k] == 0:
            gain -= 2 * prices[i - k]
        best = max(best, gain)
    
    return base + best